<a href="https://colab.research.google.com/github/aniketSanyal/OverfittingInRML/blob/main/rml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1: Setup Environment and Load Datasets

1. **Import Libraries**: We'll import PyTorch, torchvision, and other necessary libraries.
2. **Load Datasets**: Load MNIST and CIFAR10 datasets using torchvision.
3. **Data Loaders**: Create data loaders for both datasets for easy batch processing.

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Device configuration - GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST dataset
mnist_train = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
mnist_train_loader = DataLoader(dataset=mnist_train, batch_size=64, shuffle=True)
mnist_test_loader = DataLoader(dataset=mnist_test, batch_size=64, shuffle=False)

# CIFAR10 dataset
cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=True)
cifar_test = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor())
cifar_train_loader = DataLoader(dataset=cifar_train, batch_size=64, shuffle=True)
cifar_test_loader = DataLoader(dataset=cifar_test, batch_size=64, shuffle=False)


Files already downloaded and verified


### Step 2: Model Definition

We'll define simple CNN architectures suitable for each dataset. MNIST images are grayscale and smaller, while CIFAR10 images are color and larger.

#### MNIST CNN Model:
- Simple architecture with a couple of convolutional layers.

#### CIFAR10 CNN Model:
- A bit more complex due to the nature of the dataset (color images).

In [22]:
import torch.nn as nn
import torch.nn.functional as F

# CNN for MNIST
class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(7*7*64, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(x.size(0), -1) # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# CNN for CIFAR10
class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(8*8*128, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(x.size(0), -1) # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


### Step 3: FGSM Function
The FGSM method creates adversarial examples by adding a small perturbation to the original image in the direction of the gradient of the loss with respect to the input image.

### Step 4: PGD Function
PGD is a more powerful attack compared to FGSM. It applies the perturbation iteratively and projects the perturbed image back into the allowed range after each step.

In [23]:
def fgsm_attack(image, epsilon, data_grad):
    perturbed_image = image + epsilon * data_grad.sign()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

def pgd_attack(model, image, label, epsilon, alpha, iters, device):
    perturbation = torch.zeros_like(image).to(device)
    perturbation.requires_grad = True

    for _ in range(iters):
        outputs = model(image + perturbation)
        loss = F.cross_entropy(outputs, label)
        model.zero_grad()
        loss.backward()

        perturbation.data = perturbation.data + alpha * perturbation.grad.data.sign()
        perturbation.data = torch.clamp(perturbation.data, -epsilon, epsilon)

    return image + perturbation.detach()


### Step 5: Training Loop with Adversarial Training

We'll:
1. Load pretrained models or train simple models for MNIST and CIFAR10.
2. Select a few sample images from both datasets.
3. Apply FGSM and PGD attacks on these samples.
4. Visualize the results to see the effect of the attacks.

In [24]:
def train(model, device, train_loader, optimizer, epoch, attack=None, epsilon=0.01):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        if attack is not None:
            data.requires_grad = True
            output = model(data)
            loss = F.cross_entropy(output, target)
            model.zero_grad()
            loss.backward()
            data_grad = data.grad.data
            data = attack(model, data, target, epsilon, 0.01, 40, device) if attack == pgd_attack else fgsm_attack(data, epsilon, data_grad)

        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

# Example usage
model = CIFAR10_CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 11):
    train(model, device, cifar_train_loader, optimizer, epoch, attack=fgsm_attack, epsilon=0.01)
    # Optionally, test your model here


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.317297
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.953113
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.904429
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.848151
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.657304
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.747589
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.893233
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.699799
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.861703
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.482302
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.649215
Train Epoch: 2 [19200/50000 (38%)]	Loss: 1.480755
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.493898
Train Epoch: 2 [32000/50000 (64%)]	Loss: 1.501360
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.443392
Train Epoch: 2 [44800/50000 (90%)]	Loss: 1.518942
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.351961
Train Epoch: 3 [6400/50000 (13%)]	Loss: 1.581195
Train Epoch: 3 [12800/50000 (26%)]	Loss: 1.591314
Train Epoch: 3 [19200/50000 (38%)]	Loss: 1.426708
Train Epoch: 3 [25

# Result

In [27]:
def visualize(image, title):
    npimg = image.numpy()
    plt.figure(figsize=(8, 8))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.show()

# Take a sample for visualization
dataiter = iter(cifar_test_loader)
images, labels = next(dataiter)

# FGSM
images.requires_grad = True
output = model_cifar(images.to(device))
loss = F.cross_entropy(output, labels.to(device))
model_cifar.zero_grad()
loss.backward()

perturbed_data = fgsm_attack(images, 0.05, images.grad)
visualize(torchvision.utils.make_grid(perturbed_data.cpu()), "FGSM Attack on CIFAR10")

# PGD
perturbed_data_pgd = pgd_attack(model_cifar, images.to(device), labels.to(device), 0.05, 0.01, 40, device)
visualize(torchvision.utils.make_grid(perturbed_data_pgd.cpu()), "PGD Attack on CIFAR10")



NameError: name 'model_cifar' is not defined